In [17]:
# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import os 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
os.chdir('/Users/philippratz/Documents/Uni/PhD/UQAM/research/ml_time_series/code/new_push/')

In [8]:
df_ = pd.read_csv('./data/prepared_stocks.csv')

In [10]:
train_ = df_.head(2992-1000)
test_ = df_.tail(1000)

In [48]:
def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []
    i=0
    q=0.99
    errors = target - preds
    losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

In [15]:
class BaseMLP(nn.Module):
    def __init__(self, arch,
                 input_size=2,
                 output_size = 1,
                 dropout=0.1, 
                 output_function=''):
        super().__init__()
        self.output_size = output_size
        self.input_size = input_size
        self.layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        self._set_arch(arch, input_size)
        self.output_function = output_function
        
    def _set_arch(self, arch, input_size):
        current_size = input_size
        for lay_size in arch:
            self.layers.append(nn.Linear(current_size, lay_size))
            current_size = lay_size
            
        self.final_layer = nn.Linear(current_size, self.output_size)

    def forward(self, x):
        for lay_ in self.layers:
            x = F.relu(lay_(x))
            x = self.dropout(x)
            
        x = self.final_layer(x)
        
        return x

In [18]:
mlp_ = BaseMLP([4,4,4], input_size=2, output_size=2)

In [25]:
y_1 = train_.loc[:,['chf']]
y_2 = train_.loc[:,['gold']]

In [50]:
X_train = train_.loc[:, ['lag_1_chf', 'lag_1_gold']]
X_test = test_.loc[:, ['lag_1_chf', 'lag_1_gold']]

In [28]:
from torch.utils.data import Dataset

class TabularDataSet(Dataset):
    def __init__(self, X, y_1, y_2):
        self.X = X.copy()
        self.y1 = y_1
        self.y2 = y_2
        
    def __len__(self):
        return len(self.y1)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y1[idx], self.y2[idx]
    

In [43]:
data_train = TabularDataSet(X_train.to_numpy(),
                            np.float32(y_1),
                            np.float32(y_2))
    
trainloader = DataLoader(data_train, batch_size=128, shuffle=True)

In [49]:
optimizer_ = optim.Adam(mlp_.parameters(),
                        lr=0.05)

for epoch in range(50):
    for i, (x_train, y1, y2) in enumerate(trainloader):
        optimizer_.zero_grad()

        y_pred_batch = mlp_(x_train.float())

        loss_1 = quantile_loss(y_pred_batch[:,0], y1.squeeze(), 0.99)
        loss_2 = quantile_loss(y_pred_batch[:,1], y2.squeeze(), 0.99)

        total_loss = loss_1 + loss_2
        total_loss.backward()
        optimizer_.step()

In [55]:
preeds_ = mlp_(torch.from_numpy(np.float32(X_test.to_numpy())))

In [57]:
preeds_[:,]

tensor([[0.0365, 0.0440],
        [0.0365, 0.0440],
        [0.0365, 0.0440],
        ...,
        [0.0365, 0.0440],
        [0.0365, 0.0440],
        [0.0365, 0.0440]], grad_fn=<SliceBackward0>)